## 1. 파이썬과 MySQL 활용

+ 파이썬과 MySQL을 연결해주는 모듈 : mysqlclient, PyMySQL
    - pip install mysqlclient
    - pip install PyMySQL

In [1]:
! pip install mysqlclient

In [2]:
! pip install PyMySQL

In [5]:
# mysqlclient 모듈 사용
import MySQLdb

conn = MySQLdb.connect(host="127.0.0.1", user="root", password="1111", database="world")
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000275C0B10CF0>


In [6]:
# pymysql 모듈 사용
import pymysql

conn = pymysql.connect(host="127.0.0.1", user="root", password="1111", database="world")
print(conn)
conn.close()

In [9]:
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"world"}

In [13]:
conn = MySQLdb.connect(**config)
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000275C1736F20>


#### (1) SELECT

In [18]:
conn = MySQLdb.connect(**config)

sql = "select * from city limit 10"

cur = conn.cursor()
cur.execute(sql)

# print(cur)
# for data in cur:
    # print(data)
    # print("%d %s %s %s %d"% data)
    
#for a, b, c, d, e in cur:
#    print(a, b, c, d, e)

result = list()
for data in cur:
    result.append(data)
    
conn.close()

print(result)

[(1, 'Kabul', 'AFG', 'Kabol', 1780000), (2, 'Qandahar', 'AFG', 'Qandahar', 237500), (3, 'Herat', 'AFG', 'Herat', 186800), (4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800), (5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200), (6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321), (7, 'Haag', 'NLD', 'Zuid-Holland', 440900), (8, 'Utrecht', 'NLD', 'Utrecht', 234323), (9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843), (10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)]


#### (2) INSERT

In [21]:
conn = MySQLdb.connect(**config)

sql = "create table if not exists tbltest(empno  int, ename  varchar(10), sal  int,  job varchar(10))"
cur = conn.cursor()
cur.execute(sql)

sql = "insert into tbltest(empno, ename, sal) values(1, 'scott', 5000)"
cur.execute(sql)
conn.commit()

conn.close()

In [28]:
empno = 2
ename = 'miller'
sal = 1300

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values("+str(empno)+",'"+ename+"',"+str(sal)+")"
#print(sql)
cur.execute(sql)
conn.commit()

conn.close()

In [30]:
empno = 3
ename = 'tom'
sal = 2000

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
sql_data = (str(empno), ename, str(sal))

cur.execute(sql, sql_data)
conn.commit()

conn.close()

In [34]:
try :
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    empno = 4
    ename = 'jerry'
    sal = 2000
    
    sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), ename, str(sal))
    
    cur.execute(sql, sql_data)
    conn.commit()
except Exception as e:
    print("error : ", e)
finally:
    conn.close() 

In [35]:
# 데이터 여러 개 입력하기
conn = MySQLdb.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 10000)"
    cur.execute(sql)
    conn.commit()
    
conn.close()

#### (3) cursor 객체의 fetchall(), fetchmany(), fetchone()

+ default는 fetchall()

In [39]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from tbltest")
    
    #for i in cur.fetchall():
    #    print(i)
    
    #for i in cur.fetchmany(5):
    #    print(i)
    
    print(cur.fetchone())

finally:
    conn.close()

(1, 'scott', 5000, None)


#### (4) DictCursor

+ 필드명을 인덱스 대신 키로 사용

In [43]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    
    cur.execute("select * from tbltest")
    
    for i in cur.fetchall():
        # print(i[0], i[1], i[2])
        print(i["empno"], i["ename"], i["sal"])
    
finally:
    conn.close()

1 scott 5000
2 miller 1300
3 tom 2000
3 tom 2000
4 jerry 2000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000


#### (5) 응용 실습1 : CSV파일을 DB에 저장

In [ ]:
"""
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);

"""

In [53]:
import csv

file = csv.reader(open("data/input.csv"))
print(file)

# print(next(file))
# print(next(file))
# print(next(file))

header = next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("delete from suppliers")
    conn.commit()
    
    for row in file:
        # print(row)
        cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", row)
        
    conn.commit()
finally:
    conn.close()